In [23]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

df = pd.read_excel(
    "https://github.com/chris1610/pbpython/blob/master/data/salesfunnel.xlsx?raw=True"
)
mgr_options = df["Manager"].unique()

app = dash.Dash()

app.layout = html.Div([
    html.H2("Sales Funnel Report"),
    html.Div(
        [
            dcc.Dropdown(
                id="Manager",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Managers'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])


@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Manager', 'value')])
def update_graph(Manager):
    if Manager == "All Managers":
        df_plot = df.copy()
    else:
        df_plot = df[df['Manager'] == Manager]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Status"],
        values=['Quantity'],
        aggfunc=sum,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
    trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
    trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
    trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

    return {
        'data': [trace1, trace2, trace3, trace4],
        'layout':
        go.Layout(
            title='Customer Order Status for {}'.format(Manager),
            barmode='stack')
    }


if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Programmer\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2657, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1608, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'declined'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "pandas/_libs/index.pyx", line 669, in pandas._libs.index.BaseMultiIndexCodesEngine.get_loc
  File "C:\Users\Pr

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Programmer\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2657, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1608, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'declined'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "pandas/_libs/index.pyx", line 669, in pandas._libs.index.BaseMultiIndexCodesEngine.get_loc
  File "C:\Users\Programmer\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2659, in get_loc
    return self._engine.get_loc(self._maybe_c

In [24]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_table
import pandas as pd
import praw
import pandas as pd
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from config import cid, csec, uag
reddit = praw.Reddit(client_id= cid, client_secret= csec, user_agent= uag)
posts = []
new_bets = reddit.subreddit('wallstreetbets').new(limit=100)
for post in new_bets:
    posts.append([post.title, post.score, post.num_comments, post.selftext, post.created, post.pinned, post.total_awards_received])
posts = pd.DataFrame(posts,columns=['title', 'score', 'comments', 'post', 'created', 'pinned', 'total awards'])
def get_features(dataframe):
    df = posts.copy()
df['words'] = df['post'].apply(lambda x : len(x.split()))
    df['chars'] = df['post'].apply(lambda x : len(x.replace(" ","")))
    df['word density'] = (df['words'] / (df['chars'] + 1)).round(3)
    df['unique words'] = df['post'].apply(lambda x: len(set(w for w in x.split())))
    df['unique density'] = (df['unique words'] / df['words']).round(3)
return df
df = get_features(posts)
app.layout = html.Div([
    html.P(html.Button('Refresh', id='refresh'))
    ,html.P(html.Div(html.H3('Enter Subreddit')))
    ,dcc.Input(id='input-1-state', type='text', value='wallstreetbets')
    ,dash_table.DataTable(
    id='table'
    ,columns=[{"name": i, "id": i} for i in df.columns]
    ,fixed_rows={ 'headers': True, 'data': 0 }
    ,data=df.to_dict('records')
)
])
@app.callback(Output('table', 'data'),
              [Input('refresh', 'n_clicks')],
              [State('input-1-state', 'value')
               ])
def update_data(n_clicks, subreddits):
    dff = df
    if subreddits is None:
        subreddits = 'wallstreetbets'
    else:
        subreddits
    if n_clicks is None:
        raise PreventUpdate
    else:
        posts = []
        new_bets = reddit.subreddit(subreddits).hot(limit=100)
        for post in new_bets:
            posts.append([post.title, post.score, post.num_comments, post.selftext, post.created, post.pinned, post.total_awards_received])
        posts = pd.DataFrame(posts,columns=['title', 'score', 'comments', 'post', 'created', 'pinned', 'total awards'])
        dff = get_features(posts)
    
    return dff.to_dict('records')

IndentationError: unexpected indent (<ipython-input-24-4d7ab6e68177>, line 20)

In [26]:
import plotly.express as px

df = px.data.gapminder()
df_2007 = df.query("year==2007")

for template in [ "seaborn"]:
    fig = px.scatter(df_2007,
                     x="gdpPercap", y="lifeExp", size="pop", color="continent",
                     log_x=True, size_max=60,
                     template=template, title="Gapminder 2007: '%s' theme" % template)
    fig.show()